In [1]:
from kan import KAN
from sklearn.model_selection import KFold,cross_val_score
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
import torch
import process as data
import numpy as np
import scipy.stats.stats as st
import csv
import pandas as pd

ac_p,label=data.deal()
aac=data.fe()
ctd=data.CTD()
gaac=data.gaac()


39


In [2]:
protein_list = []
with open('sequence.txt', 'r') as file:
        for line in file:
            line = line.strip()
            if not line.startswith(">"):  # 忽略ID行，只保存序列行
                protein_list.append(line)


In [3]:
def maccs_feature(protein_list):
    maccs_list=[]
    with open('maccs_fp.txt','r') as f:
        for line in f:
            maccs=list(map(int,line.strip().split()))
            print(maccs)
            maccs_list.append(maccs)
    return maccs_list
maccs_list=maccs_feature(protein_list)
           
# len(maccs_list),len(maccs_list[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [4]:
def calculate_GAAC(protein_list):
    # 定义氨基酸分组
    groups = {
        'G1': 'GAVLMI',  # 脂肪族
        'G2': 'FYW',     # 芳香族
        'G3': 'KRH',     # 带正电
        'G4': 'DE',      # 带负电
        'G5': 'STCPNQ'   # 不带电
    }

    # 初始化结果列表
    results = []

    # 遍历蛋白质序列
    for protein in protein_list:
        # 计算每个组的频率
        group_counts = {key: 0 for key in groups}
        total_length = len(protein)

        for aa in protein:
            for group_name, group_aa in groups.items():
                if aa in group_aa:
                    group_counts[group_name] += 1
                    break

        # 计算频率
        group_frequencies = [count / total_length for count in group_counts.values()]
        results.append(group_frequencies)

    return results

# 示例蛋白质序列列表

gaac = calculate_GAAC(protein_list)


In [5]:
def calculate_aac(protein_list):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

    # 初始化存储结果的列表
    aac_list = []

    for protein_sequence in protein_list:
        # 初始化AAC特征字典
        aac = {aa: 0 for aa in amino_acids}

        # 计算每种氨基酸在序列中的频率
        for aa in protein_sequence:
            if aa in aac:
                aac[aa] += 1

        # 将频率转换为比例
        sequence_length = len(protein_sequence)
        aac = [count / sequence_length for aa, count in aac.items()]

        # 将结果添加到列表中
        aac_list.append(aac)

    return aac_list
aac = calculate_aac(protein_list)

In [6]:
def aaindex():
    protein_list = []
    csv_file_path = 'aaindex1.csv'
    lag=2
    data = []
    with open(csv_file_path, mode='r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        
        # Skip the first cell of the first row
        for row in csv_reader:
            # Append the row with the first cell removed to the data list
            # data.append([float(value) for value in row[1:]])
            data.append(row[1:])
    data=data[1:]
    AAindex_list = [
        [float(value) if value not in ['NA', ''] else None for value in row]
        for row in data
    ]
    # NAKH900113 【200】，KRIW710101【146】， HUTJ700103【117】，ZIMJ680103【399】，TANS770104【368】，CEDJ970105【459】，QIAN880127【283】，LEVM760107【158】
    # selected_indices = [200, 146, 117, 399, 368,459,283, 158]
    # AAindex_list = [AAindex_list[i] for i in selected_indices if i < len(AAindex_list)]
    with open('sequence.txt', 'r') as file:
        for line in file:
            line = line.strip()
            if not line.startswith(">"):  # 忽略ID行，只保存序列行
                protein_list.append(line)
                
    autocorrelation = []
    for sequence in protein_list:
        temp=[]
        for property_values in AAindex_list:
        # 将氨基酸序列转换为属性值序列
        #     property_values = np.array([property_dict[aa] for aa in sequence])
            property_values = [0 if value is None else value for value in property_values]
            # 计算属性值的平均值
            # print(property_values)
            mean_value = np.mean(property_values)
            # print(mean_value)
            # 计算Moran自相关
            n = len(sequence)
            autocorr = np.sum((property_values[:-lag] - mean_value) * (property_values[lag:] - mean_value))
            autocorr /= (n - lag)
            temp.append(autocorr)

        autocorrelation.append(temp)
        
    v = []
    for i in range(len(autocorrelation)):
        vtar = autocorrelation[i]
        vtarv = []
        vtar7 = 0
        vtar8 = 0
        vtar9 = 0
        s = pd.Series(vtar)
        vtar3 = np.mean(vtar)  # These 4 dimensions are relevant statistical terms
        vtar4 = st.kurtosis(vtar)
        vtar5 = np.var(vtar)
        vtar6 = st.skew(vtar)
        #for p in range(len(vtar)): # These 3 dimensions are inspired by PAFIG algorithm
        #vtar7=vtar[p]**2+vtar7
        #if vtar[p]>va:
        #vtar8=vtar[p]**2+vtar8
        #else:
        #vtar9=vtar[p]**2+vtar9
        vcf1 = []
        vcf2 = []
        for j in range(len(vtar) - 1):  #Sequence-order-correlation terms
            vcf1.append((vtar[j] - vtar[j + 1]))
        for k in range(len(vtar) - 2):
            vcf2.append((vtar[k] - vtar[k + 2]))
        vtar10 = np.mean(vcf1)
        vtar11 = np.var(vcf1)
        vtar11A = st.kurtosis(vcf1)
        vtar11B = st.skew(vcf1)
        vtar12 = np.mean(vcf2)
        vtar13 = np.var(vcf2)
        vtar13A = st.kurtosis(vcf2)
        vtar13B = st.skew(vcf2)
        vtarv.append(vtar3)
        vtarv.append(vtar4)
        vtarv.append(vtar5)
        vtarv.append(vtar6)
        #vtarv.append(vtar7/len(vtar))
        #vtarv.append(vtar8/len(vtar))
        #vtarv.append(vtar9/len(vtar))
        vtarv.append(vtar10)
        vtarv.append(vtar11)
        vtarv.append(vtar11A)
        vtarv.append(vtar11B)
        vtarv.append(vtar12)
        vtarv.append(vtar13)
        vtarv.append(vtar13A)
        vtarv.append(vtar13B)
        v.append(vtarv)
    return v



In [7]:
v=aaindex()
v = np.array(v)  # 转换为 NumPy 数组
# print(v.shape,type(v))
# 检查是否为二维数组
if len(v.shape) == 2:
    # 对每一行进行 Z-score 标准化
    v = (v - np.mean(v, axis=1, keepdims=True)) / np.std(v, axis=1, keepdims=True)
    print(v[0])
else:
    print("v 不是二维数组")

[-0.55786107 -0.55561036  0.93206    -0.55761631 -0.55751747  1.97661393
 -0.55618619 -0.557518   -0.55751747  2.105045   -0.55637066 -0.55752139]


In [8]:
X=np.concatenate((aac,maccs_list,v),axis=1)
print(X.shape)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

model = KAN(width=[37,3,1], grid=3, k=3)
fold = 1

y=label
y=np.array(y)
X = X.astype(np.float32)
y = y.astype(np.float32)
X.shape,y.shape,X[0],y[0]

(740, 199)
checkpoint directory created: ./model
saving model version 0.0


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)
X_train=np.array(X_train)
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train).view(-1, 1)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test).view(-1, 1)
X_train = torch.tensor(X_train ,dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.float32)
X_test = torch.tensor(X_test,dtype=torch.float32)
y_test = torch.tensor(y_test,dtype=torch.float32)
X_train.shape,y_train.shape,X_train[0],y_test[0]

/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == "":
/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/home/wangli/anaconda3/envs/mykan/lib/python3.7/

In [12]:
dataset = {}
dataset = {
    'train_input': X_train,
    'test_input': X_test,
    'train_label': y_train,
    'test_label': y_test
}
X_train

tensor([[ 0.1000,  0.0000,  0.0000,  ...,  2.1050, -0.5568, -0.5573],
        [ 0.0000,  0.0000,  0.0000,  ...,  2.1050, -0.5569, -0.5572],
        [ 0.0667,  0.0000,  0.0000,  ...,  2.1050, -0.5569, -0.5572],
        ...,
        [ 0.1000,  0.0000,  0.0333,  ...,  2.1050, -0.5563, -0.5576],
        [ 0.0000,  0.1538,  0.0000,  ...,  2.1050, -0.5570, -0.5572],
        [ 0.3333,  0.0000,  0.0000,  ...,  2.1050, -0.5568, -0.5572]])

In [ ]:
# 5,7,9
model = KAN(width=[199,5,1], grid=7, k=7, seed=42) 
from sklearn.metrics import roc_auc_score, confusion_matrix, matthews_corrcoef
def train_acc():
    pred=model(X_train)
    with torch.no_grad():
        pred_labels = (pred > 0.5).float() 
        auc = roc_auc_score(y_train.cpu(), pred.cpu())

    # 混淆矩阵计算
        tn, fp, fn, tp = confusion_matrix(y_train.cpu(), pred_labels.cpu()).ravel()

        # Specificity (SP)
        sp = tn / (tn + fp) if (tn + fp) > 0 else 0
        
        acc = (tp + tn) / (tp + tn + fp + fn)

        # Sensitivity (SN)
        sn = tp / (tp + fn) if (tp + fn) > 0 else 0

        # Matthews Correlation Coefficient (MCC)
        mcc = matthews_corrcoef(y_train.cpu(), pred_labels.cpu())
    print(f"ACC: {acc:.4f}, AUC: {auc:.4f}, SP: {sp:.4f}, MCC: {mcc:.4f}, SN: {sn:.4f}")
    return torch.mean((torch.round(model(X_train)[:, 0]) == y_train[:, 0]).float())

def test_acc():
    pred=model(X_test)
    with torch.no_grad():
        pred_labels = (pred > 0.5).float() 
        auc = roc_auc_score(y_test.cpu(), pred.cpu())

    # 混淆矩阵计算
        tn, fp, fn, tp = confusion_matrix(y_test.cpu(), pred_labels.cpu()).ravel()

        # Specificity (SP)
        sp = tn / (tn + fp) if (tn + fp) > 0 else 0
        
        acc = (tp + tn) / (tp + tn + fp + fn)

        # Sensitivity (SN)
        sn = tp / (tp + fn) if (tp + fn) > 0 else 0

        # Matthews Correlation Coefficient (MCC)
        mcc = matthews_corrcoef(y_test.cpu(), pred_labels.cpu())
    print(f"ACC: {acc:.4f}, AUC(test): {auc:.4f}, SP: {sp:.4f}, MCC: {mcc:.4f}, SN: {sn:.4f}")
    return torch.mean((torch.round(model(X_test)[:, 0]) == y_test[:, 0]).float())

# results = model.train(dataset, opt="LBFGS", steps=10, metrics=(train_acc, test_acc)) ,lamb=0.001 lamb_entropy=4.,lamb=0.1,lamb_l1=2.5,
# lamb=0.005 train/fit
results = model.fit(dataset, opt="LBFGS", steps=30, metrics=(train_acc, test_acc));
# print(results['train_acc'][-1], results['test_acc'][-1])

In [ ]:
# MLP
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50,50)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, alpha=1e-4,
                    solver='sgd', verbose=10, random_state=1,
                    learning_rate_init=.1)
mlp.fit(X_train, y_train)
predictions_train = mlp.predict(X_train)
predictions_test = mlp.predict(X_test)

# 计算准确率
train_accuracy = accuracy_score(y_train, predictions_train)
test_accuracy = accuracy_score(y_test, predictions_test)

print(f'Train Accuracy: {train_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

In [ ]:
# LSTM


In [ ]:
# SVM
import numpy as np
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

model = SVC(kernel='linear', probability=True)  # probability=True为了后续计算AUC
X_train,X_test=X_test,X_train
y_train,y_test=y_test,y_train
# 训练模型
model.fit(X_train, y_train)

# 预测
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# 计算准确度
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# 计算AUC
y_train_prob = model.predict_proba(X_train)[:, 1]
y_test_prob = model.predict_proba(X_test)[:, 1]
train_auc = roc_auc_score(y_train, y_train_prob)
test_auc = roc_auc_score(y_test, y_test_prob)

# 输出结果
print(f"Training Accuracy: {train_accuracy:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Training AUC: {train_auc:.2f}")
print(f"Test AUC: {test_auc:.2f}")

In [36]:
def calculate_GAAC(protein_list):
    # 定义氨基酸分组
    groups = {
        'G1': 'GAVLMI',  # 脂肪族
        'G2': 'FYW',     # 芳香族
        'G3': 'KRH',     # 带正电
        'G4': 'DE',      # 带负电
        'G5': 'STCPNQ'   # 不带电
    }

    # 初始化结果列表
    results = []

    # 遍历蛋白质序列
    for protein in protein_list:
        # 计算每个组的频率
        group_counts = {key: 0 for key in groups}
        total_length = len(protein)
        
        for aa in protein:
            for group_name, group_aa in groups.items():
                if aa in group_aa:
                    group_counts[group_name] += 1
                    break
        
        # 计算频率
        group_frequencies = {key: count / total_length for key, count in group_counts.items()}
        results.append(group_frequencies)

    return results

# 示例蛋白质序列列表
protein_list = ["GAVLMIKRHDE", "STCPNQFYWG", "GAVLMIKRHDEGAVLMIKRHDE"]
gaac_features = calculate_GAAC(protein_list)

for i, features in enumerate(gaac_features):
    print(f"Protein {i + 1}: {features}")


/home/wangli/anaconda3/envs/transformer-env/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/home/wangli/anaconda3/envs/transformer-env/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == "":
/home/wangli/anaconda3/envs/transformer-env/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/home/wangli/anaco